## detect Diabetic Retinopathy using AI
### Capstone[](http://) Group:
### Grade 12
### Red Sea STEM

In [ ]:
from fastai.vision import *
import os

In [ ]:
# this is where we will copy our pretrained models
os.makedirs('/root/.cache/torch/checkpoints')

In [ ]:
!cp ../input/resnet34/resnet34.pth /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth

In [ ]:
# to save the models
model_path = 'models'

# to save the plk models
plk_path = 'plk'

# to save the plots
plot_path = 'plots'

if not os.path.exists(model_path):
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, plot_path))
    os.makedirs(os.path.join(model_path, plk_path))

In [ ]:
'''
Severity Levels

0 - 'No_DR',
1 - 'Mild',
2 - 'Moderate',
3 - 'Severe',
4 - 'Proliferate_DR'
'''

classes = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

In [ ]:
path = Path('../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/')
path.ls()

In [ ]:
'''
Remove the images that we cannot open. 
Execute this only once per kernel run.
'''
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train='.', valid_pct=0.2, 
                                  ds_tfms=get_transforms(), size=224, 
                                  num_workers=4, bs=16).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(10, 7))

In [ ]:
learn = cnn_learner(data, models.resnet34, 
                    metrics=[accuracy, error_rate], 
                    model_dir='/kaggle/working/models')

In [ ]:
torch.cuda.is_available()

In [ ]:
learn.fit_one_cycle(70)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.save('colored_stage1', return_path=True)

In [ ]:
learn.export(os.path.abspath('./models/plk/colored_stage1.pkl'))

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20, max_lr=slice(1e-5, 1e-4))

In [ ]:
learn.save('colored_stage2', return_path=True)

In [ ]:
learn.export(os.path.abspath('./models/plk/colored_stage2.pkl'))

In [ ]:
learn.load('colored_stage2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()
plt.savefig('models/plots/interp.png')
plt.show()